**PanNuke Dataset Segmentation**

6 Masks 
* 0: Neoplastic cells
* 1: Inflammatory 
* 2: Connective/Soft tissue cells
* 3: Dead Cells
* 4: Epithelial 
* 6: Background

**Train, Test, Val Paths**

In [ ]:
train_images_path = '/kaggle/input/pannuke-images-dataset/Pan Nuke - Reduced/train/images/'
train_masks_path = '/kaggle/input/pannuke-images-dataset/Pan Nuke - Reduced/train/masks/'

test_images_path = '/kaggle/input/pannuke-images-dataset/Pan Nuke - Reduced/test/images/'
test_masks_path = '/kaggle/input/pannuke-images-dataset/Pan Nuke - Reduced/test/masks/'

val_images_path = '/kaggle/input/pannuke-images-dataset/Pan Nuke - Reduced/val/images/'
val_masks_path = '/kaggle/input/pannuke-images-dataset/Pan Nuke - Reduced/val/masks/'

**Importing Libraries**

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Conv2DTranspose, UpSampling2D, Concatenate, BatchNormalization, Activation, Add
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.losses import Loss
from keras.backend import epsilon
from tensorflow import reduce_sum as sum
from tensorflow.keras.utils import plot_model
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

**Train, Test, Val Images Names List**

In [ ]:
# train images
train_image_names = os.listdir(train_images_path)[:2000]

# test images
test_image_names = os.listdir(test_images_path)[:1000]

# val images
val_image_names = os.listdir(val_images_path)[:1000]

**Display Images**

In [ ]:
labels = {0: "Neoplastic cells",
1: "Inflammatory",
2: "Connective/Soft tissue cells",
3: "Dead Cells",
4: "Epithelial",
5: "Background"}

In [ ]:
# function to display predicted images
def display(images):
    # loop through each set of images
    for i in range(images.shape[0]):
        # create a subplot for each set of images
        fig, axes = plt.subplots(1, images.shape[-1], figsize=(20, 4))
        
        k = 0  
        
        # loop through each image in the set
        for j in range(images.shape[-1]):
            # display the image on the corresponding subplot
            axes[j].imshow(images[i, :, :, j], cmap='gray')
            axes[j].axis('off') 
            axes[j].set_title(f'{labels[k]}')
            k += 1
        
        plt.show()

In [ ]:
# display single mask of 256x256x6
def display_mask(image):
    plt.figure(figsize=(15, 10))
    for i in range(image.shape[-1]):
        plt.subplot(2, 3, i+1)
        plt.imshow(image[:, :, i],cmap="gray")
        plt.title(f'Channel {i}')
        plt.axis('off')
    plt.show()

In [ ]:
# display single image
def display_image(image, text=None, channels=1):
    if channels == 1:
        plt.imshow(image, cmap='gray')
    else:
        plt.imshow(image)

    if text:
        plt.text(0, 0, text, color='white', fontsize=12, ha='left', va='top', bbox=dict(facecolor='black', alpha=0.5))

    plt.axis('off')
    plt.show()

In [ ]:
# display images before and after normalization
def display_images(images_path, original_image_names, normalized_images, num_images_to_display=1):
    
    # just to display original images
    original_images = []
    
    for img in train_image_names[:num_images_to_display]:
        image_path = os.path.join(images_path, img)
        image = cv2.imread(image_path)
        original_images.append(image)

    plt.figure(figsize=(12, 6))

    # display images before normalization
    for i in range(num_images_to_display):
        # Display original image
        plt.subplot(2, num_images_to_display, i+1)
        plt.imshow(original_images[i])
        plt.axis('off')

        # display normalized image
        plt.subplot(2, num_images_to_display, num_images_to_display+i+1)
        plt.imshow(normalized_images[i], cmap="gray")
        plt.axis('off')

    plt.tight_layout()
    plt.show()

**Z-Score Normalization**

In [ ]:
# normalizing using z-score
def z_score_normalization(image):
    
    image = image.astype(np.float32)

    # calculating mean
    mean = np.mean(image, axis=(0, 1))

    # calculating std
    std = np.std(image, axis=(0, 1))

    # normalization function
    # epsilon is added to avoid dividing by zero
    normalized_image = (image - mean) / (std + 1e-7)

    # convert to uint8 for image
    normalized_image = normalized_image.astype(np.uint8)

    return normalized_image

**Contours**

In [ ]:
def contours(grayscale_image):

    # apply Canny edge detection to find edges
    edges = cv2.Canny(grayscale_image, 30, 150)

    # find contours
    contours, _ = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # draw contours on the original image
    contour_image = np.zeros_like(grayscale_image)
    cv2.drawContours(contour_image, contours, -1, (255, 255, 255), 1)

    return contour_image

**CLAHE**

In [ ]:
def clahe(image):
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_channel, a, b = cv2.split(lab)

    # applying CLAHE to L-channel
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(16,16))
    cl = clahe.apply(l_channel)

    # merge the CLAHE enhanced L-channel with the a and b channel
    limg = cv2.merge((cl,a,b))

    # converting image from LAB Color model to BGR color spcae
    enhanced_img = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
        
    return enhanced_img

**Preprocessing Images**

In [ ]:
# preprocessing images
def preprocess_images(image_names, images_path):

    # creating empty list to store normalized images
    normalized_images = []

    for filename in image_names:

        # read image
        image_path = os.path.join(images_path, filename)
        image = cv2.imread(image_path)
        
        # applying clahe
        clahe_image = clahe(image)

        # apply z-score normalization
        normalized_image = z_score_normalization(clahe_image)
        
        # converting to grayscale image
        #grayscale_image = cv2.cvtColor(normalized_image, cv2.COLOR_BGR2GRAY)
        
        # applying contours
        #contour_image = contours(grayscale_image)

        # append normalized image to the list
        normalized_images.append(normalized_image)

    return normalized_images

**Applying Preprocessing on Images**

**Testing Preprocessing**

In [ ]:
img_path = os.path.join(train_images_path, train_image_names[3])
img = cv2.imread(img_path)
display_image(img, text="Original Image", channels=3)
clahe_img = clahe(img)
display_image(clahe_img, text="CLAHE Image", channels=3)
normalized_img = z_score_normalization(clahe_img)
display_image(normalized_img, text="Normalized Image", channels=3)
grayscale_img = cv2.cvtColor(normalized_img, cv2.COLOR_BGR2GRAY)
display_image(grayscale_img, text="Grayscale Image", channels=1)
contour_img = contours(grayscale_img)
display_image(contour_img, text="Contour Image", channels=1)

In [ ]:
# preprocess training images
train_normalized_images = preprocess_images(train_image_names, train_images_path)

# preprocess test images
test_normalized_images = preprocess_images(test_image_names, test_images_path)

# preprocess validation images
val_normalized_images = preprocess_images(val_image_names, val_images_path)

# just to display first 5 images
train_imgs = []


# display images
# display first five
n= 5
display_images(train_images_path, train_image_names, train_normalized_images, n)

**Preprocessing Masks**

In [ ]:
# getting masks of each image
def get_masks(image_names, mask_names):

    # creating empty list to store lists of masks for each image
    masks =  []

    for filename in image_names:

        # extract image name without extension
        image_name = os.path.splitext(filename)[0]

        # creating empty list to store 6 masks of each image
        m = []

        for i in range(1, 7):
            mask_path = os.path.join(mask_names, f"{image_name}.jpeg_mask_{i}.png")
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

            # apply threshold to convert grayscale mask image to binary
            _, binary_mask = cv2.threshold(mask, 0, 255, cv2.THRESH_BINARY)

            m.append(binary_mask)

        masks.append(m)

    return masks

In [ ]:
# combining masks of each image
def combine_masks(mask_lists):
    
    combined_masks = []

    for masks in mask_lists:
        # stack the masks along a new axis to create a single image with six channels
        combined_mask = np.stack(masks, axis=-1)

        # append the combined mask to the list
        combined_masks.append(combined_mask)

    return combined_masks

**Calling Functions to Get Masks and Combine them**

In [ ]:
# get train masks
train_masks = get_masks(train_image_names, train_masks_path)

# combined train masks
combined_train_masks = combine_masks(train_masks)

In [ ]:
# get test masks
test_masks = get_masks(test_image_names, test_masks_path)

# combined test masks
combined_test_masks = combine_masks(test_masks)

In [ ]:
# get val masks
val_masks = get_masks(val_image_names, val_masks_path)

# combined val masks
combined_val_masks = combine_masks(val_masks)

**Normalized Image and its Masks**

In [ ]:
display_image(train_normalized_images[3], text="Normalized Image", channels=1)

In [ ]:
display_mask(combined_train_masks[3])

**Converting Numpy arrays to Tensors**

In [ ]:
# tensor of train images
train_normalized_images = tf.convert_to_tensor(train_normalized_images)

In [ ]:
# tensor of test images
test_normalized_images = tf.convert_to_tensor(test_normalized_images)

In [ ]:
# tensor of val images
val_normalized_images = tf.convert_to_tensor(val_normalized_images)

In [ ]:
# tensor of combined test masks
combined_train_masks = tf.convert_to_tensor(combined_train_masks)

In [ ]:
# tensor of combined test masks
combined_test_masks = tf.convert_to_tensor(combined_test_masks)

In [ ]:
# tensor of combined val masks
combined_val_masks = tf.convert_to_tensor(combined_val_masks)

**Dice Loss**

In [ ]:
class DiceLoss(tf.keras.losses.Loss):
    def __init__(self, epsilon=1e-6, name='dice_loss'):
        super().__init__(name=name)
        self.epsilon = epsilon

    def call(self, y_true, y_pred):
        dice_losses = []
        for i in range(y_true.shape[-1]):
            intersection = tf.reduce_sum(y_true[..., i] * y_pred[..., i])
            dice_coeff = (2. * intersection + self.epsilon) / (tf.reduce_sum(y_true[..., i]) + tf.reduce_sum(y_pred[..., i]) + self.epsilon)
            dice_losses.append(1. - dice_coeff)
        return tf.reduce_mean(dice_losses)

**U-Net Model with Resnet Encoder**

In [ ]:
!pip install git+https://github.com/qubvel/segmentation_models

In [ ]:
os.environ['SM_FRAMEWORK'] = 'tf.keras'

In [ ]:
import segmentation_models as sm


BACKBONE = 'resnet34'
#preprocess_input = sm.get_preprocessing(BACKBONE)

model = sm.Unet(BACKBONE, classes=6, activation="softmax")

In [ ]:
# getting the last layer to modify
output_layer = model.layers[-1]

# modifying last layer
model.layers[-1] = Conv2D(6, (1, 1), activation='softmax', name='output_layer')

model.compile(
    loss=DiceLoss(),
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()

**Train Model**

In [ ]:
model.fit(
   x=train_normalized_images,
   y=combined_train_masks,
   batch_size=16,
   epochs=10,
   validation_data=(val_normalized_images, combined_val_masks),
)

**Evaluate Model**

In [ ]:
# evaluate on test set
test_loss, test_dice = model.evaluate(test_normalized_images, combined_test_masks)
print(f'Test Loss: {test_loss:.4f}, Test Dice Coefficient: {test_dice:.4f}')

**Pipleline**

In [ ]:
def pipeline(image_names, images_path):
    norm_images = preprocess_images(image_names, images_path)
    #display_images(images_path, image_names, norm_images, len(image_names))
    img = tf.convert_to_tensor(norm_images)
    pred = model.predict(img)
    display(pred)

**Predicted Images**

In [ ]:
images_path = test_images_path
predicted_image_names= os.listdir(images_path)[12:17]

pipeline(predicted_image_names, images_path)

**Predicted Images Names**

In [ ]:
predicted_image_names

**Ground Truth**

In [ ]:
image_path = os.path.join(test_images_path, predicted_image_names[1])
image = cv2.imread(image_path)
display_image(image, text=None, channels=3)

In [ ]:
test_masks = get_masks(image_names, test_masks_path)
groundTruth_test_masks = combine_masks(test_masks)
display_mask(groundTruth_test_masks[1])